In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score
import xgboost as xgb
from scipy.stats import uniform, randint

#### Read in Dataset

In [25]:
df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/feature_engineered_data.csv', 
                 encoding='utf-8')

C:\Users\sega9\AppData\Local\Temp\ipykernel_26272\1272201554.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/feature_engineered_data.csv',


In [26]:
# List of columns to drop
# columns_to_drop = ['title','body','address','latitude','longitude','geometry']
# dogs_allowed, pets_allowed
# has_parking
columns_to_drop = ["id",'title', 'body','fee', 'has_photo', 'address', 'cityname', 'latitude',
       'longitude', 'source', 'geometry', 'description_len', 'cats_allowed','state',
       'has_Fireplace', 'has_Gym', 'has_Pool',
       'has_Storage', 'has_Wood_Floors', 'has_Gated', 'has_Dishwasher',
       'has_Clubhouse', 'has_Refrigerator', 'has_Elevator', 'has_Patio_Deck',
       'has_Cable_or_Satellite', 'has_Garbage_Disposal', 'has_Internet_Access',
       'has_Luxury', 'has_Playground', 'has_Basketball', 'has_Hot_Tub',
       'has_Tennis', 'has_Doorman', 'has_TV', 'has_Washer_Dryer', 'has_View',
       'has_Alarm', 'has_Golf', 'has_AC']
# Drop the specified columns
df = df.drop(columns=columns_to_drop)
df = df.rename(columns={"dogs_allowed": "pets_allowed"})
df = df.rename(columns={"has_Parking": "has_parking"})

In [27]:
df.dtypes

bathrooms        float64
bedrooms         float64
price            float64
square_feet        int64
zipcode          float64
POPULATION       float64
POP_SQMI         float64
SQMI             float64
perc_sub25k      float64
perc_25-50k      float64
perc_50-75k      float64
perc_75-100k     float64
perc_100-200k    float64
perc_abv200k     float64
pets_allowed       int64
has_parking        int64
dtype: object

#### Encode Data for Modeling

In [28]:
X = df.drop(['price'], axis=1)  # 'price' is the dependent variable
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Linear Regession

In [29]:
# Create and fit the linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)

R-squared: 0.4368470645191712


#### XGBoost

In [30]:
# Create and fit the XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_test)

# Calculate R-squared
r_squared = r2_score(y_test, y_pred)
print("R-squared:", r_squared)

R-squared: 0.77054847470075


#### Use Model to Predict Prices of ZIP Permutaions

In [31]:
perms = pd.read_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/permTESTfinal.csv', 
                 encoding='utf-8',
                 dtype={'zipcode': object})

In [32]:
perms.dtypes

zipcode           object
perc_sub25k      float64
perc_25-50k      float64
perc_50-75k      float64
perc_75-100k     float64
perc_100-200k    float64
perc_abv200k     float64
STATE_x           object
POPULATION       float64
POP_SQMI         float64
SQMI             float64
beds               int64
baths              int64
square-feet        int64
STATE             object
has_parking        int64
pets_allowed       int64
dtype: object

In [33]:
perms.drop('STATE_x', axis=1, inplace=True)
perms.rename(columns={'baths': 'bathrooms', 
                   'beds': 'bedrooms',
                   'square-feet': 'square_feet'
                   }, inplace=True)

In [34]:
perms = perms[['bathrooms', 'bedrooms', 'square_feet', 'zipcode', 'POPULATION',
       'POP_SQMI', 'SQMI', 'perc_sub25k', 'perc_25-50k', 'perc_50-75k',
       'perc_75-100k', 'perc_100-200k', 'perc_abv200k','pets_allowed', 'has_parking']]

In [35]:
perms.columns

Index(['bathrooms', 'bedrooms', 'square_feet', 'zipcode', 'POPULATION',
       'POP_SQMI', 'SQMI', 'perc_sub25k', 'perc_25-50k', 'perc_50-75k',
       'perc_75-100k', 'perc_100-200k', 'perc_abv200k', 'pets_allowed',
       'has_parking'],
      dtype='object')

In [36]:
X.columns

Index(['bathrooms', 'bedrooms', 'square_feet', 'zipcode', 'POPULATION',
       'POP_SQMI', 'SQMI', 'perc_sub25k', 'perc_25-50k', 'perc_50-75k',
       'perc_75-100k', 'perc_100-200k', 'perc_abv200k', 'pets_allowed',
       'has_parking'],
      dtype='object')

In [37]:
perms.head()


,bathrooms,bedrooms,square_feet,zipcode,POPULATION,POP_SQMI,SQMI,perc_sub25k,perc_25-50k,perc_50-75k,perc_75-100k,perc_100-200k,perc_abv200k,pets_allowed,has_parking
0,1,1,250,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0
1,1,1,500,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0
2,1,1,750,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0
3,1,1,1000,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0
4,1,1,1250,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0


In [38]:
# xgb_model.predict(perms)

tempPerms = perms.copy()
tempPerms["zipcode"] = tempPerms[["zipcode"]].apply(pd.to_numeric)
xgb_model.predict(tempPerms)

array([1327.8805, 1372.4633, 1473.346 , ..., 2453.5518, 2671.8032,
       2937.181 ], dtype=float32)

In [39]:
# Predict on the entire dataset
tempPerms = perms.copy()
tempPerms["zipcode"] = tempPerms[["zipcode"]].apply(pd.to_numeric)

perms['predicted_price'] = xgb_model.predict(tempPerms)

In [40]:
perms.head()

,bathrooms,bedrooms,square_feet,zipcode,POPULATION,POP_SQMI,SQMI,perc_sub25k,perc_25-50k,perc_50-75k,perc_75-100k,perc_100-200k,perc_abv200k,pets_allowed,has_parking,predicted_price
0,1,1,250,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0,1327.880493
1,1,1,500,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0,1372.463257
2,1,1,750,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0,1473.345947
3,1,1,1000,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0,1594.752197
4,1,1,1250,01001,16979.0,1405.55,12.08,0.272532,0.246781,0.182403,0.114807,0.153433,0.030043,0,0,1342.541016


In [41]:
perms.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/permsPredictedFIN.csv'
             , encoding='utf-8',index= False)


### AVG Predicted for each State/ZIP

In [21]:
irs_data = pd.read_csv("C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataSource/20zpallagi.csv",
                       dtype={'zipcode': object})
irs_data = irs_data[['STATE', 'zipcode']] #Relevant cols

zip_sum = irs_data.groupby(['zipcode','STATE']).count().reset_index() # Get count of returns by zip code
perms2 = pd.merge(perms, zip_sum, on='zipcode', how='left')

In [22]:
avgState = perms2.groupby(['STATE'])['predicted_price'].mean().reset_index()
avgState

,STATE,predicted_price
0,AK,2077.432617
1,AL,1435.323730
2,AR,1322.914551
3,AZ,1547.960938
4,CA,3205.934814
5,CO,1821.142578
6,CT,2992.862793
7,DC,2841.562256
8,DE,1719.618774
9,FL,1904.595947


In [ ]:
avgState.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/permsStates.csv'
             ,encoding='utf-8',index= False)

In [26]:
avgZIP = perms2.groupby(['zipcode','STATE'])['predicted_price'].mean().reset_index()
avgZIP

,zipcode,STATE,predicted_price
0,01001,MA,2430.899414
1,01002,MA,2766.367676
2,01005,MA,2404.767822
3,01007,MA,2277.391113
4,01008,MA,2455.345459
...,...,...,...
27637,99824,AK,5020.646484
27638,99827,AK,1803.437256
27639,99833,AK,1868.733765
27640,99835,AK,1793.891724


In [28]:
avgZIP.to_csv('C:/Users/sega9/Documents/GA Tech/2024.Spring/CSE 6242/Group Project/ApartmentRent/DataModified/permsZIPs.csv'
             ,encoding='utf-8',index= False)